In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib as mpl
from sys import argv
import os
os.chdir('/home/tapas/som-tsp/')

In [2]:
def select_closest(candidates,origin):
    return euclidean_distance(candidates,origin).argmin()

In [3]:
def euclidean_distance(a,b):
    return np.linalg.norm(a - b, axis = 1)

In [4]:
def route_distance(best):
    points = cities[['x','y']]
    distance = euclidean_distance(points,np.roll(points,1,axis = 0))
    return np.sum(distances)

In [5]:
def generate_network(size):
    return np.random.rand(size=2)

In [6]:
def get_neighbourhood(center,radix,domain):
    if radix <1:
        radix = 1
    deltas = np.absoulte(center - np.arrange(domain))
    distances = np.minimum(deltas,domain-deltas)
    return np.exp(-(distances * distances)/(2 * (radix * radix)))

In [7]:
def get_route(cities,network):
    cities['winner'] = cities[['x','y']].apply(
           lambda c: select_closest(network,c),
            axis = 1, raw = True)
    return cities.sort_values('winner').index

In [8]:
def read_tsp(filename):
    with open(filename) as f:
        node_coord_start = None
        dimension = None
        lines = f.readlines()
        i = 0
        while not dimension or not node_coord_start:
            lines = lines[i]
            if lines.startwith('DIMENSION :'):
                dimension = int(line.split()[-1])
            if line.startwith('NODE_COORD_SECTION'):
                node_coord_start = i
            i = i + 1
        print('Problem with {} cities read.'.format(dimension))
        f.seek(0)
        cities = pd.read_csv(
              f,
              skip_rows = node_coord + 1,
              sep = '',
              names = ['city','y','x'],
              dtype = {'city':str,'x':np.float64,'y':np.float64},
              header =  None,
              nrows = dimension
               )
        return cities

In [9]:
def normalize(points):
    ratio = (points.x.max() - points.x.min())/(points.y.max() - points.y.min()),1
    ratio = np.array(ratio) / max(ratio)
    norm = points.apply(lambda c: (c - c.min()) / (c.max() - c.min()))
    return norm.apply(lambda p: ratio * p, axis = 1)

In [10]:
def plot_network(cities,neurons, name = 'diagram.png',ax = None):
    mpl.rcParams['agg.path.chunksize'] = 10000
    if not ax:
        fig = plt.figure(figsize = (5,5), frameon = False)
        axis = fig.add_axes([0,0,1,1])
        axis.set_aspect('equal',adjustable = 'datalim')
        plt.axis('off')
        axis.scatter(cities['x'],cities['y'],color = 'red',s=4)
        axis.plot(neurons[:,0],neurons[:,1],'r',ls='-',color='#0063ba',markersize=2)
        plt.savefig(name,bbox_inches = 'tight',pad_inches = 0,dpi=200)
        plt.close()
    else:
        ax.scatter(cities['x'],cities['y'],color='red',s=4)
        ax.plot(neurons[:,0],neurons[:,1],'r',ls='-',color='00#63ba',marksize=2)
        return ax

In [11]:
def plot_route(cities,route,diagram='diagram.png',ax=None):
    mpl.rcParams['agg.path.chunksize'] = 10000
    if not ax:
        fig = plt.figure(figsize=(5,5), frameon=False)
        axis = fig.add_axes([0,0,1,1])
        axis.set_aspect('equal',adjustable='datalim')
        plt.axis('off')
        axis.scatter(cities['x'],cities['y'],color='red',s=4)
        route = cities.reindex(route)
        route.loc[route.shape[0]] = route.iloc[0]
        axis.plot(route['x'],route['y'],color='purple',linewidth=1)
        plt.savefig(name,bbox_inches='tight',pad_inches=0,dpi=200)
        plt.close()
    else:
        ax.scatter(cities['x'],cities['y'],color='red',s=4)
        route = cities.reindex(route)
        route.loc[route.shape[0]] = route.iloc[0]
        ax.plot(route['x'],route['y'],color='purple',linewidth=1)
        return ax

In [20]:
def main():
    if len(argv) != 2:
        print("Correct use: Python src/main.py <filename>.tsp")
        return -1
    problem = read_tsp('/home/tapas/som-tsp/assets/fi10639.tsp')
    route = som(problem,10000)
    problem = problem.reindex(route)
    distance = route_distance(problem)
    print('Route found of length{}'.format(distance))

In [21]:
def som(problem, iterations, learning_rate=0.8):
    """Solve the TSP using a Self-Organizing Map."""

    # Obtain the normalized set of cities (w/ coord in [0,1])
    cities = problem.copy()

    cities[['x', 'y']] = normalize(cities[['x', 'y']])

    # The population size is 8 times the number of cities
    n = cities.shape[0] * 8

    # Generate an adequate network of neurons:
    network = generate_network(n)
    print('Network of {} neurons created. Starting the iterations:'.format(n))

    for i in range(iterations):
        if not i % 100:
            print('\t> Iteration {}/{}'.format(i, iterations), end="\r")
        # Choose a random city
        city = cities.sample(1)[['x', 'y']].values
        winner_idx = select_closest(network, city)
        # Generate a filter that applies changes to the winner's gaussian
        gaussian = get_neighborhood(winner_idx, n//10, network.shape[0])
        # Update the network's weights (closer to the city)
        network += gaussian[:,np.newaxis] * learning_rate * (city - network)
        # Decay the variables
        learning_rate = learning_rate * 0.99997
        n = n * 0.9997

        # Check for plotting interval
        if not i % 1000:
            plot_network(cities, network, name='diagrams/{:05d}.png'.format(i))

        # Check if any parameter has completely decayed.
        if n < 1:
            print('Radius has completely decayed, finishing execution',
            'at {} iterations'.format(i))
            break
        if learning_rate < 0.001:
            print('Learning rate has completely decayed, finishing execution',
            'at {} iterations'.format(i))
            break
    else:
        print('Completed {} iterations.'.format(iterations))

    plot_network(cities, network, name='diagrams/final.png')

    route = get_route(cities, network)
    plot_route(cities, route, 'diagrams/route.png')
    return route

In [22]:
if __name__ == '__main__':
    main()

Correct use: Python src/main.py <filename>.tsp
